[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [3]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import trim_messages
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

from operator import itemgetter
from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)


In [9]:
## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    start_on="human",    
)

## 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt 
    | llm
)

## 7. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)


In [10]:
## 멀티턴으로 실행

while True : 
    query = input('육아휴직과 관련된 질문하세요(S 입력시 종료) >> ')
    if query.upper() == "S":
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )
    print('\n' + '*' *50)

육아휴직의 기간은 1년 이내로 하며, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.

1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모
2. 「한부모가족지원법」 제4조제1호의 부 또는 모
3. 고용노동부령으로 정하는 장애아동의 부 또는 모

해당 규정은 남녀고용평등법 제19조 제2항에 명시되어 있습니다.
**************************************************
육아휴직은 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청할 수 있으며, 사업주는 이를 허용해야 합니다. 육아휴직의 기간은 기본적으로 1년 이내로 하며, 특정 조건을 충족하는 경우 6개월 이내에서 추가로 사용할 수 있습니다. 육아휴직을 이유로 해고하거나 불리한 처우를 해서는 안 되며, 육아휴직 후에는 휴직 전과 같은 업무 또는 같은 수준의 임금을 지급받는 직무로 복귀해야 합니다. 또한, 육아휴직 기간은 근속기간에 포함됩니다. (남녀고용평등법 제19조)
**************************************************
육아휴직 대상자는 임신 중인 여성 근로자 또는 만 8세 이하 또는 초등학교 2학년 이하의 자녀(입양한 자녀 포함)를 양육하기 위한 근로자입니다. 이들은 육아휴직을 신청할 수 있으며, 사업주는 이를 허용하여야 합니다. 다만, 대통령령으로 정하는 예외적인 경우에는 허용하지 않을 수 있습니다. (남녀고용평등법 제19조 제1항).
**************************************************
"남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"에서는 육아휴직 수당의 구체적인 금액에 대해 명시하지 않지만, 육아휴직을 사용하는 근로자의 생계비용과 사업주의 고용유지비용의 일부를 국가가 지원할 수 있도록 규정하고 있습니다(제20조). 육아휴직 수당의 구체적인 금액 및 지급 방법은 별도의 규정이나 법령에 

In [11]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간은 1년 이내로 하며, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.\n\n1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모\n2. 「한부모가족지원법」 제4조제1호의 부 또는 모\n3. 고용노동부령으로 정하는 장애아동의 부 또는 모\n\n해당 규정은 남녀고용평등법 제19조 제2항에 명시되어 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run--f3bde76e-4786-4e3f-b128-7a102f6e7363-0'), HumanMessage(content='육아휴직 조건은?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직은 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청할 수 있으며, 사업주는 이를 허용해야 합니다. 육아휴직의 기간은 기본적으로 1년 이내로 하며, 특정 조건을 충족하는 경우 6개월 이내에서 추가로 사용할 수 있습니다. 육아휴직을 이유로 해고하거나 불리한 처우를 해서는 안 되며, 육아휴직 후에는 휴직 전과 같은 업무 또는 같은 수준의 임금을 지급받는 직무로 복귀해야 합니다. 또한, 육아휴직 기간은 근속기간에 포함됩니다. (남녀고용평등법 제19조)', additional_kwargs={}, response_metadata={'finish_reason': 'stop

In [12]:
for r in store['abc'].messages:
    print(f"[{r.type.upper()}] {r.content}")

[HUMAN] 육아휴직 기간은?
[AI] 육아휴직의 기간은 1년 이내로 하며, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다.

1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우의 부 또는 모
2. 「한부모가족지원법」 제4조제1호의 부 또는 모
3. 고용노동부령으로 정하는 장애아동의 부 또는 모

해당 규정은 남녀고용평등법 제19조 제2항에 명시되어 있습니다.
[HUMAN] 육아휴직 조건은?
[AI] 육아휴직은 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청할 수 있으며, 사업주는 이를 허용해야 합니다. 육아휴직의 기간은 기본적으로 1년 이내로 하며, 특정 조건을 충족하는 경우 6개월 이내에서 추가로 사용할 수 있습니다. 육아휴직을 이유로 해고하거나 불리한 처우를 해서는 안 되며, 육아휴직 후에는 휴직 전과 같은 업무 또는 같은 수준의 임금을 지급받는 직무로 복귀해야 합니다. 또한, 육아휴직 기간은 근속기간에 포함됩니다. (남녀고용평등법 제19조)
[HUMAN] 육아휴직 대상자는?
[AI] 육아휴직 대상자는 임신 중인 여성 근로자 또는 만 8세 이하 또는 초등학교 2학년 이하의 자녀(입양한 자녀 포함)를 양육하기 위한 근로자입니다. 이들은 육아휴직을 신청할 수 있으며, 사업주는 이를 허용하여야 합니다. 다만, 대통령령으로 정하는 예외적인 경우에는 허용하지 않을 수 있습니다. (남녀고용평등법 제19조 제1항).
[HUMAN] 육아휴직 수당은 얼마야?
[AI] "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률"에서는 육아휴직 수당의 구체적인 금액에 대해 명시하지 않지만, 육아휴직을 사용하는 근로자의 생계비용과 사업주의 고용유지비용의 일부를 국가가 지원할 수 있도록 규정하고 있습니다(제20조). 육아휴직 수당의 구체적인 금액 및 지급 방법은 별도의 규정이나 법령에 따릅니다. 이를 보다 자세히 알아보시려면 관련 부처인 고용노동부나 해당 기관의 구체적인 가이드라인을 